# **Introduction**

By Yvtsan Levy 

Misdiagnosis of the many diseases impacting agricultural crops can lead to misuse of chemicals leading to the emergence of resistant pathogen strains, increased input costs, and more outbreaks with significant economic loss and environmental impacts. Current disease diagnosis based on human scouting is time-consuming and expensive, and although computer-vision based models have the promise to increase efficiency, the great variance in symptoms due to age of infected tissues, genetic variations, and light conditions within trees decreases the accuracy of detection.

The purpose of the project is to diagnose apple tree diseases solely based on leaf images. The data is a set of images that is sapareted to those categories: "healthy", "scab", "rust", and "multiple diseases". Solving this problem is important because diagnosing plant diseases early can save tonnes of agricultural produce every year. This will benefit not only the general population by reducing hunger, but also the farmers by ensuring they get the harvest they deserve.

The project is based on the data set "[Plant Pathology 2020 - FGVC7](https://www.kaggle.com/c/plant-pathology-2020-fgvc7/overview/description)" Contains 3642 images of apple leaves divided into training data and test data.

# **Domain knowledge**

Global human population growth amounts to around 83 million annually, and as the population  grows, so does the global demand for food is rising. In order to meet the demand, agricultural productivity must increase. One of the factors for decrease in crop yield are plants diseases, and identifing them is one of the bottle necks in the process of treating them. Current disease diagnosis based on human scouting is time-consuming and expensive. Diagnosing plants diseases using computer-vision based models can improve the efficeincy of those processes and increace crop yield.

Apple tree are one of the most common fruit trees, and worldwide production of apples in 2018 was 86 million tonnes. 2 of the most common diseases of apple trees are apple scab and rust. Apple scab is a common disease of plants in the rose family (Rosaceae) that is caused by the ascomycete fungus Venturia inaequalis. Although apple scab rarely kills its host, infection typically leads to fruit deformation and premature leaf and fruit drop. The reduction of fruit quality and yield may result in crop losses of up to 70%. Rusts are plant diseases caused by pathogenic fungi of the order Pucciniales. Rusts are considered among the most harmful pathogens to agriculture, horticulture and forestry. Rust fungi are major concerns and limiting factors for successful cultivation of agricultural and forest crops.

add image of rust and scabs***


# **Project's Target**


Our goal is to produce a model that classify correcly the health of a leaf and can say in great confidence what disease does it have. We will use images of apple leaves that are divided into 4 classes: healty leaves,leaves with scab,leaves with rust, and leaves with scab and rust. Our data-base contains 3642 images of apple leaves, 1821 classified images and 1821 images with no classification. We will try to train a model that can classify correcly images of apple leaves to those classes.

# **Install and Import Necessary Libraries**

Importing the libraries that we will use

In [ ]:
!pip install -q efficientnet
import os
import numpy as np
import pandas as pd
import seaborn as sns
import cv2
import matplotlib.pyplot as plt
import matplotlib.pyplot
%matplotlib inline
import sklearn
import sklearn.cluster
import plotly.graph_objects as go
import keras
from sklearn.preprocessing import MultiLabelBinarizer
import tensorflow as tf
import tensorflow.keras.layers as L
import tensorflow.keras.backend as K
from tensorflow.keras.preprocessing import image
import imblearn.over_sampling
import warnings
from tqdm.notebook import tqdm
tqdm.pandas()


from glob import glob
import itertools
import warnings
warnings.filterwarnings("ignore")

#**Data Loading and Constant Defining** 

Firstly we will load our data and define constants

In [ ]:
IMAGE_PATH = '../input/plant-pathology-2021-fgvc8/train_images/'
TRAIN_PATH = '../input/plant-pathology-2021-fgvc8/train.csv'
valid_path = '../input/plant-pathology-2021-fgvc8/test_images'
seed=1994

train_data = pd.read_csv(TRAIN_PATH)

In [ ]:
# useful for getting number of files
image_files = glob(IMAGE_PATH + '/*.jp*g')
valid_image_files = glob(valid_path + '/*/*.jp*g')

In [ ]:
# useful for getting number of classes
folders = glob(TRAIN_PATH + '/*')

In [ ]:
# look at an image for fun
plt.imshow(image.load_img(np.random.choice(image_files)));

In [ ]:
#lets count the instances of each class we have :

fig,ax=plt.subplots(figsize=(16,8))
sns.countplot(train_data['labels'])
#rotate labels
plt.setp(ax.get_xticklabels(),rotation=45)

plt.title('Label counts')

In [ ]:
#converting the labels as multiple labels:
train_data['labels']=train_data['labels'].str.split(' ')

mlb = MultiLabelBinarizer()

# one hot encode labels
lab=mlb.fit_transform(train_data['labels'])
lab[:10]

In [ ]:
train_data[:10]

In [ ]:
#classes for OHE encoded var.
classes=mlb.classes_
classes

In [ ]:
train_data['image']

In [ ]:
DF = pd.DataFrame(
    data=lab,
    columns=classes
)

print(DF.head(),DF.shape)

In [ ]:
DF['image'] =  train_data['image']
print(DF.head(),DF.shape)

In [ ]:
train_data = DF

#**Functions**

We will use utility functions several times in our project, so we will define them here. Most of them are used to process images and reshape them or the data-sets that contins them

In [ ]:
#prints a image
# gets an image to prints
# returns nothing
def show_image(img):
  fig = plt.imshow(img)

#loads an image and returns it 
# gets the image id of the image we wants to load from the raw images and the shape of the output image
# returns an image with the shape of image_size
def load_image(image_id,image_size=(100,100)):
    file_path = image_id
    image = cv2.imread(IMAGE_PATH + file_path,1)
    image = cv2.resize(image, image_size)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return image

#loads an image from returns it 
# gets the image id of the image we wants to load, the path to the dictunary and the shape of the output image
# returns an image with the shape of image_size
def load_dif_image(image_id,path,image_size=(100,100)):
    file_path = image_id 
    image = cv2.imread(path + file_path,1)
    image = cv2.resize(image, image_size)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return image

#creat from image objects list a reshaped 4-dimentional array(image_number,row,column,color_channel)
# get a list of image objects
# returns a 4-dimentional np array of the same images
def images_4d_array(files_list):
  images_list = [img[np.newaxis, :, :, :3] for img in files_list]
  images_array = np.vstack(images_list)
  return (images_array)

# retrive a list of images that fullfil the cond and show up to 9 pics of them. cond is a health situation name name. 
# gets health situation name:'scab','rust','multiple_diseases','healthy'
# returns a list of images that fullfil the condition, and prints up to 9 images that fullfil them
def show_cond(cond):
  cond_list=train_images[train_data[cond]==1]
  cols, rows = 3,3  
  fig, ax = plt.subplots(nrows=rows, ncols=cols, figsize=(15, rows*10/3))
  for i in range(cols*rows):
    if(i>=cond_list.size):
      break
    ax[int(i/cols), int(i%rows)].imshow(cond_list.iloc[i])
  plt.show()
  return (cond_list)

# retrive a list of images that fullfil the cond. cond is a health situation name name. 
# gets health situation name:'scab','rust','multiple_diseases','healthy'
# returns a list of images that fullfil the condition
def get_cond(cond):
  cond_list=train_images[train_data[cond]==1] 
  return (cond_list)

# calculate the avarge per color channel
# gets a 4-dimensional  array and an int  
# returns the avarage value of the cells by the 4th dimension channel position 
def get_average_channel(x, channel):
    return x[:,:,:,channel].mean(axis = (1, 2)).reshape(-1, 1)

# claculate the avarge per color channel for x
# gets image 4-dimentional array 
# returns array of 
def get_channels(x):
    return np.hstack([get_average_channel(x, i) for i in range(3)])

# reshapes the 4-dimentional array into 2-dimentional array
# gets a 4-dimentional array
# returns a 2-dimentional array
def get_all_pixels(x):
    return x.reshape(-1, np.prod(x.shape[1:]))

# merges images to one image
# gets an 4-dimentional array and the numbers of images to print as number per rows and number of images per row
# returns an image that is a marge of the other ones
def merge_images(image_batch, size = [20, 20]):
    h,w = image_batch.shape[1], image_batch.shape[2]
    c = image_batch.shape[3]
    img = np.zeros((int(h*size[0]), w*size[1], c))
    for idx, im in enumerate(image_batch):
        i = idx % size[1]
        j = idx // size[1]
        img[j*h:j*h+h, i*w:i*w+w,:] = im/255
    return img

# get the data and creates a numerated classes array
# gets the images data
# returns an array of numerated classes
def get_target_array(data):
  targets=[]
  for index, row in data.drop('image',axis=1).iterrows():
    if (row[0]==1):
      targets.append(0)
    elif (row[1]==1):
      targets.append(1)
    elif (row[2]==1):
      targets.append(2)
    elif (row[3]==1):
      targets.append(3)
    elif (row[4]==1):
      targets.append(4)
    elif (row[5]==1):
      targets.append(5)
  return (np.array(targets))


# get the data and creates a numerated classes array
# gets the images data
# returns an array of numerated classes
def get_target_array_no_im_id(data):
  targets=[]
  for index, row in data.iterrows():
    if (row[0]==1):
      targets.append(0)
    elif (row[1]==1):
      targets.append(1)
    elif (row[2]==1):
      targets.append(2)
    elif (row[3]==1):
      targets.append(3)
    elif (row[4]==1):
      targets.append(4)
    elif (row[5]==1):
      targets.append(5)
  return (np.array(targets))

# **Load Images**

Thus far, we talk about how our images fit in our classes, but we haven't has a look at them, so we shall do it now.

In [ ]:
n_sample = 1500

In [ ]:
#loads all of the images
# train_images = train_data["image"].progress_apply(load_image, args=((100,100),))

#load a random sample
train_images = train_data["image"].sample(n = n_sample, random_state = seed).progress_apply(load_image)

#Build the dataset of current images
current_train_data=train_data.loc[train_images.index]
targets=get_target_array(current_train_data)

In [ ]:
current_train_data=train_data.loc[train_images.index]
targets=lab

# **Data Exploration and Analysis**

As a first step with our data, should explore our data and check what classes and data we have, the number of classes and the amount of images can tell us what dificulties can occur further in the project

Lets print our data of the images

In [ ]:
print(train_data.head(),train_data.shape)

As can be seen, we have a list of images and their class out of 4, healthy, multiple_diseases, rust and scab. Every entry is an image and have the value 1 for bieng in the class and 0 for not.


We have 18632 images, lets find the distribution of the data in the classes.

As can be seen, 3 of the 4 classes have similar representation in our sata set. The multiple_diseases class is highly underrepresented, we have less then 100 images of this class, out of 18632 images. Our data is imbalanced, and we have to remember it as we draw conclusions

## **First Look at Our Images**

Thus far, we talk about how our images fit in our classes, but we haven't has a look at them, so we shall do it now.

First, lest see some leaves with scab

In [ ]:
scab_list=show_cond('scab') 

As can be seen from the images, scabs manifest itself as dark brown spots on the leaves. one can easily think that those are just dirty leaves, but a professional will know that they have scab.

Lest see some leaves with rust

In [ ]:
rust_list=show_cond('rust')

As can be seen from the images, rust manifest itself as yellow-brownish spots on the leaves. The spots are much more identifiable and noticeable.

Lest see some leaves with complex

In [ ]:
multi_list=show_cond('complex')

As can be seen from the images, those leaves have features of both diseases.

Lets see some healthy leaves

In [ ]:
healthy_list=show_cond('healthy')

AS we expected the leaves has no features of the diseases

Lets see some frog_eye_leaf_spot leaves

In [ ]:
frog_eye_list=show_cond('frog_eye_leaf_spot')

Lets see some powdery_mildew leaves

In [ ]:
powdery_mildew_list=show_cond('powdery_mildew')

## **Color Analysis**

Colors in pictures are the basic features, thus analyising them is our first step. We already saw that the dieases has characteristics that are connected to color, so it is naturally a good direction.

Changing the images array to 4 dimentional (image number, row, column, color channel) array so we could analyze them 

In [ ]:
healthy_array=images_4d_array(healthy_list)
scab_array=images_4d_array(scab_list)
multi_array=images_4d_array(multi_list)
rust_array=images_4d_array(rust_list)
frog_eye_array=images_4d_array(frog_eye_list)
powdery_mildew_array=images_4d_array(powdery_mildew_list)

print(healthy_array.shape,scab_array.shape,multi_array.shape,rust_array.shape,frog_eye_array.shape,powdery_mildew_array.shape)

 Lets plot the color histograms for each class

In [ ]:
#print histograms of colors
def plot_hist_normed(images, channel, col):
    vals = images[:,:,:,channel].flatten()
    matplotlib.pyplot.ylabel(col)
    matplotlib.pyplot.hist(vals)
    matplotlib.pyplot.yticks([])


#red  
matplotlib.pyplot.figure(figsize =(30, 15))
matplotlib.pyplot.subplot(3, 6, 1)
plot_hist_normed(healthy_array, 0, 'red')
matplotlib.pyplot.title('healthy leaves')
matplotlib.pyplot.subplot(3, 6, 2)
plot_hist_normed(scab_array, 0, 'red')
matplotlib.pyplot.title('scab leaves')
matplotlib.pyplot.subplot(3, 6, 3)
plot_hist_normed(rust_array, 0, 'red')
matplotlib.pyplot.title('rust leaves')
matplotlib.pyplot.subplot(3, 6, 4)
plot_hist_normed(multi_array, 0, 'red')
matplotlib.pyplot.title('multiple diseases leaves')
matplotlib.pyplot.subplot(3, 6, 5)
plot_hist_normed(powdery_mildew_array, 0, 'red')
matplotlib.pyplot.title('powdery_mildew leaves')
matplotlib.pyplot.subplot(3, 6, 6)
plot_hist_normed(frog_eye_array, 0, 'red')
matplotlib.pyplot.title('frog_eye_array leaves')

#green
matplotlib.pyplot.subplot(3, 6, 7)
plot_hist_normed(healthy_array, 1, 'green')
matplotlib.pyplot.title('healthy leaves')
matplotlib.pyplot.subplot(3, 6, 8)
plot_hist_normed(scab_array, 1, 'green')
matplotlib.pyplot.title('scab leaves')
matplotlib.pyplot.subplot(3, 6, 9)
plot_hist_normed(rust_array, 1, 'green')
matplotlib.pyplot.title('rust leaves')
matplotlib.pyplot.subplot(3, 6, 10)
plot_hist_normed(multi_array, 1, 'green')
matplotlib.pyplot.title('multiple diseases leaves')
matplotlib.pyplot.subplot(3, 6, 11)
plot_hist_normed(powdery_mildew_array, 1, 'green')
matplotlib.pyplot.title('powdery_mildew_array leaves')
matplotlib.pyplot.subplot(3, 6, 12)
plot_hist_normed(frog_eye_array, 1, 'green')
matplotlib.pyplot.title('frog_eye_ leaves')

#blue
matplotlib.pyplot.subplot(3, 6, 13)
plot_hist_normed(healthy_array, 2, 'blue')
matplotlib.pyplot.title('healthy leaves')
matplotlib.pyplot.subplot(3, 6, 14)
plot_hist_normed(scab_array, 2, 'blue')
matplotlib.pyplot.title('scab leaves')
matplotlib.pyplot.subplot(3, 6, 15)
plot_hist_normed(rust_array, 2, 'blue')
matplotlib.pyplot.title('rust leaves')
matplotlib.pyplot.subplot(3, 6, 16)
plot_hist_normed(multi_array, 2, 'blue')
matplotlib.pyplot.title('multiple diseases leaves')
matplotlib.pyplot.subplot(3, 6, 17)
plot_hist_normed(powdery_mildew_array, 2, 'blue')
matplotlib.pyplot.title('powdery_mildew_array leaves')
matplotlib.pyplot.subplot(3, 6, 18)
plot_hist_normed(frog_eye_array, 2, 'blue')
matplotlib.pyplot.title('frog_eye leaves')


matplotlib.pyplot.show()

It is not very informative, it is hard to notice the differences as the images are mostly similar. maybe we sould check the median and the mean of each channel

Lest plot the mean and median values for the classes

In [ ]:
def summary(images, channel, col):
    vals = images[:,:,:,channel].flatten()
    chan_mean = np.mean(vals)
    chan_median = np.median(vals)
    print('{} mean: {}, median: {}'.format(col, str(chan_mean), str(chan_median)))

print('healthy leaves:')
summary(healthy_array, 0, 'red')
summary(healthy_array, 1, 'green')
summary(healthy_array, 2, 'blue')
print('scab leaves:')
summary(scab_array, 0, 'red')
summary(scab_array, 1, 'green')
summary(scab_array, 2, 'blue')
print('rust leaves:')
summary(rust_array, 0, 'red')
summary(rust_array, 1, 'green')
summary(rust_array, 2, 'blue')
print('multiple diseases leaves:')
summary(multi_array, 0, 'red')
summary(multi_array, 1, 'green')
summary(multi_array, 2, 'blue')
print('powdery_mildew leaves:')
summary(powdery_mildew_array, 0, 'red')
summary(powdery_mildew_array, 1, 'green')
summary(powdery_mildew_array, 2, 'blue')
print('frog_eye leaves:')
summary(frog_eye_array, 0, 'red')
summary(frog_eye_array, 1, 'green')
summary(frog_eye_array, 2, 'blue')


As can be seen the blue channels mean and median of the diseased leaves are lower, and the red channels mean and median are higher then the healthy ones. We should explore the difference further by comparing the healty and diseased leaves pictures

In [ ]:
diseased_leaves_array=np.concatenate((rust_array,multi_array,scab_array,frog_eye_array,powdery_mildew_array))
diseased_leaves_array.shape

In [ ]:
#red  
matplotlib.pyplot.figure(figsize =(30, 15))
matplotlib.pyplot.subplot(3, 2, 1)
plot_hist_normed(healthy_array, 0, 'red')
matplotlib.pyplot.title('healthy leaves')
matplotlib.pyplot.subplot(3, 2, 2)
plot_hist_normed(diseased_leaves_array, 0, 'red')
matplotlib.pyplot.title('diseased leaves')

#green
matplotlib.pyplot.subplot(3, 2, 3)
plot_hist_normed(healthy_array, 1, 'green')
matplotlib.pyplot.title('healthy leaves')
matplotlib.pyplot.subplot(3, 2, 4)
plot_hist_normed(diseased_leaves_array, 1, 'green')
matplotlib.pyplot.title('diseased leaves')

#blue
matplotlib.pyplot.subplot(3, 2, 5)
plot_hist_normed(healthy_array, 2, 'blue')
matplotlib.pyplot.title('healthy leaves')
matplotlib.pyplot.subplot(3, 2, 6)
plot_hist_normed(diseased_leaves_array, 2, 'blue')
matplotlib.pyplot.title('diseased leaves')

matplotlib.pyplot.show()


print('healthy leaves:')
summary(healthy_array, 0, 'red')
summary(healthy_array, 1, 'green')
summary(healthy_array, 2, 'blue')
print('diseased leaves:')
summary(diseased_leaves_array, 0, 'red')
summary(diseased_leaves_array, 1, 'green')
summary(diseased_leaves_array, 2, 'blue')

As we expected the blue values are lower and the red values are higher. We can see that the diseased leaves have brown or yellow spots on them. That might be the reason for the discrepancy as in those colors the blue channel has a lower value and the red color has a higher value, and if that is the reason, those differences might help us to identify the healthy leaves. This hypothesis is bold and in order to try and see if it has any truth in it more exploration is needed


We want to further explore the main difference between the classes, and to try and check our hypothesis. Principal component analysis (PCA) is an effective way to do so. PCA is a method to reduce the dimension of our features, and find the most dominant ones 

# **Modeling**

In this section we will try to fit models to our data so we could predict on new data, which of the 4 classes they are part of.

## **CNN-Convolution Neural Network**

Convolution neural network is the fittest and most robust model that we know of to this task. It will serve as our main tool in this classification task, thus we will prepare the ground before applying it. A convolutional neural network consists of an input and an output layer, as well as multiple hidden layers. The hidden layers of a CNN typically consist of a series of convolutional layers that convolve with a multiplication or other dot product. The activation function is commonly a ReLU layer, and is subsequently followed by additional convolutions such as pooling layers, fully connected layers and normalization layers, referred to as hidden layers because their inputs and outputs are masked by the activation function and final convolution.

### **Set Early Stopping Parameters**

We have limited resources, and we want to make the most out of them. In order to do so, we set stopping parameters which determands that as the model reaching a plateau, the run will stop

In [ ]:
LR_reduce=keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy',
                            factor=.5,
                            patience=10,
                            min_lr=.000001,
                            verbose=0)

ES_monitor=keras.callbacks.EarlyStopping(monitor='val_loss',
                          patience=20)


reg = .0005

### **Plot History Function**

We will use this function to plot our model progression

In [ ]:
def plot_history(history):

  h = history.history

  offset = 5
  epochs = range(offset, len(h['loss']))

  matplotlib.pyplot.figure(1, figsize=(20, 6))

  matplotlib.pyplot.subplot(121)
  matplotlib.pyplot.xlabel('epochs')
  matplotlib.pyplot.ylabel('loss')
  matplotlib.pyplot.plot(epochs, h['loss'][offset:], label='train')
  matplotlib.pyplot.plot(epochs, h['val_loss'][offset:], label='val')
  matplotlib.pyplot.legend()

  matplotlib.pyplot.subplot(122)
  matplotlib.pyplot.xlabel('epochs')
  matplotlib.pyplot.ylabel('accuracy')
  matplotlib.pyplot.plot(h[f'accuracy'], label='train')
  matplotlib.pyplot.plot(h[f'val_accuracy'], label='val')
  matplotlib.pyplot.legend()

  matplotlib.pyplot.show()

  pred_test = model.predict(x_val)
  roc_sum = 0
  classes=['healthy',	'complex',	'rust',	'scab' , 'frog_eye_leaf_spot' , 'powdery_mildew']
  for i in range(6):
      score = sklearn.metrics.roc_auc_score(y_val.iloc[:,i].values.astype('int32'), pred_test[:,i])
      roc_sum += score
      print(f'AUC-ROC {classes[i]}  {score:.3f}')

  roc_sum /= 6
  print(f'totally roc score:{roc_sum:.3f}')

### **CNN Model**

Now we will create a CNN model to our use, it is based on several standards CNN we found and combined

In [ ]:
def creat_model(img_size):
  model = keras.models.Sequential()

  model.add(keras.layers.Conv2D(32, kernel_size=(5,5),activation='relu', input_shape=(img_size, img_size, 3), kernel_regularizer=keras.regularizers.l2(reg)))
  model.add(keras.layers.BatchNormalization(axis=-1,center=True,scale=False))
  model.add(keras.layers.Conv2D(128, kernel_size=(5,5),activation='relu', kernel_regularizer=keras.regularizers.l2(reg)))
  model.add(keras.layers.BatchNormalization(axis=-1,center=True,scale=False))
  model.add(keras.layers.MaxPooling2D(pool_size=(2,2), padding='SAME'))
  model.add(keras.layers.Dropout(.25))

  model.add(keras.layers.Conv2D(32, kernel_size=(3,3),activation='relu', kernel_regularizer=keras.regularizers.l2(reg)))
  model.add(keras.layers.BatchNormalization(axis=-1,center=True,scale=False))
  model.add(keras.layers.Conv2D(128, kernel_size=(3,3),activation='relu',kernel_regularizer=keras.regularizers.l2(reg)))
  model.add(keras.layers.BatchNormalization(axis=-1,center=True,scale=False))
  model.add(keras.layers.MaxPooling2D(pool_size=(2,2), padding='SAME'))
  model.add(keras.layers.Dropout(.25))


  model.add(keras.layers.Conv2D(128, kernel_size=(5,5),activation='relu', kernel_regularizer=keras.regularizers.l2(reg)))
  model.add(keras.layers.BatchNormalization(axis=-1,center=True,scale=False))
  model.add(keras.layers.Conv2D(512, kernel_size=(5,5),activation='relu',kernel_regularizer=keras.regularizers.l2(reg)))
  model.add(keras.layers.BatchNormalization(axis=-1,center=True,scale=False))
  model.add(keras.layers.MaxPooling2D(pool_size=(2,2), padding='SAME'))
  model.add(keras.layers.Dropout(.25))

  model.add(keras.layers.Conv2D(128, kernel_size=(3,3),activation='relu',kernel_regularizer=keras.regularizers.l2(reg)))
  model.add(keras.layers.BatchNormalization(axis=-1,center=True,scale=False))
  model.add(keras.layers.Conv2D(512, kernel_size=(3,3),activation='relu',kernel_regularizer=keras.regularizers.l2(reg)))
  model.add(keras.layers.BatchNormalization(axis=-1,center=True,scale=False))
  model.add(keras.layers.MaxPooling2D(pool_size=(2,2), padding='SAME'))
  model.add(keras.layers.Dropout(.25))

  model.add(keras.layers.Flatten())
  model.add(keras.layers.Dense(300,activation='relu'))
  model.add(keras.layers.BatchNormalization(axis=-1,center=True,scale=False))
  model.add(keras.layers.Dropout(.25))
  model.add(keras.layers.Dense(200,activation='relu'))
  model.add(keras.layers.BatchNormalization(axis=-1,center=True,scale=False))
  model.add(keras.layers.Dropout(.25))
  model.add(keras.layers.Dense(100,activation='relu'))
  model.add(keras.layers.BatchNormalization(axis=-1,center=True,scale=False))
  model.add(keras.layers.Dropout(.25))
  model.add(keras.layers.Dense(6,activation='softmax'))

  model.summary()
 
  
  return model

### **First Attempt- 100x100 Image Size**


First we will try our model on images that have 100 pixels as their heigth and 100 pixels are their lenght

Lets load the images

In [ ]:
# y_val

In [ ]:
# train_100X100_images = train_data["image"].progress_apply(load_image, args=((100,100),))


Now we will splits the data into train and test data

In [ ]:
# targets=train_data.drop('image',axis=1)

In [ ]:
# train_data

In [ ]:
# current_train_data=train_data.loc[train_100X100_images.index]
# images_array=images_4d_array(train_100X100_images)
# targets=current_train_data.drop('image',axis=1)
# x_train, x_val, y_train, y_val = sklearn.model_selection.train_test_split(images_array, targets, test_size=0.2, random_state=seed)
# x_train = x_train.astype('float32')
# x_val = x_val.astype('float32')
# x_train /= 255
# x_val /= 255
# x_train.shape, x_val.shape, y_train.shape, y_val.shape

Running the model on 100x100 images

In [ ]:
# model = creat_model(100)
# model.compile(optimizer='rmsprop',
#               loss='categorical_crossentropy',
#               metrics=['accuracy']
#               )

In [ ]:
# history = model.fit(x_train,
#                     y_train,
#                     batch_size=24,
#                     epochs=100,
#                     steps_per_epoch=x_train.shape[0] // 24,
#                     verbose=0,
#                     callbacks=[ES_monitor,LR_reduce],
#                     validation_data=(x_val, y_val),
#                     validation_steps=x_val.shape[0]//24
#                     )

Lets print our results

In [ ]:
# score = model.evaluate(x_val, y_val, verbose=0)

# print('Test loss:', score[0])
# print('Test accuracy:', score[1])
# plot_history(history)

Our accuracy has improved drastically to 84%, but there are still some problems, we can see at the end that there is a gap between the validation and the train matrices, which can indicate that there is some kind of overfitting. We can also see that the model is having hard time classifing the "multiple_diseases" class, due to it lower AUC-ROC score.






Lets take a look at the confusin matrix for more insight

In [ ]:
# y_val_pred = model.predict_classes(x_val)
# mat = sklearn.metrics.confusion_matrix(get_target_array_no_im_id(y_val),y_val_pred)
# sns.heatmap(mat.T, square=True, annot=True, cbar=False)
# matplotlib.pyplot.xlabel('true label')
# matplotlib.pyplot.ylabel('predict label');

The classes are ordered in this manner: "healthy"  "multiple_diseases"  "rust"  "scab". We can see that our model is pretty good at identifing the "rust" class and that it is over all can identify the classes we a decent accuracy. We can see that the main issue of our model is it poor prediction of the "multiple_diseases" class, it could be that the under representation of that class is a big factor of that. We can that our model predicts some leaves with "scab" as "healthy", we think that it can mistake the rust to be the leaves veins. We think that maybe using better resolution of the images can help that problems.

In [ ]:
# model.save("..output/kaggle/working/model_100X100.h5")

### **Second Attempt- 224x224 Image Size**


We will try to use better resolution so the model could maybe classify some of the classes ("scab") better 

In [ ]:
#In order to save memory in kaggle I resized it to 100 by 100
# img_size = 224
img_size = 100

In [ ]:
train_224X224_images = train_data["image"].progress_apply(load_image, args=((img_size,img_size),))

Now we will splits the data into train and test data

In [ ]:
current_train_data=train_data.loc[train_224X224_images.index]
images_array=images_4d_array(train_224X224_images)
targets=current_train_data.drop('image',axis=1)
x_train2, x_val, y_train2, y_val = sklearn.model_selection.train_test_split(images_array, targets, test_size=0.2, random_state=seed)
x_train2 = x_train2.astype('float32')
x_val = x_val.astype('float32')
x_train2 /= 255
x_val /= 255
x_train2.shape, x_val.shape, y_train2.shape, y_val.shape

Running the model on 224x224 images

In [ ]:
model = creat_model(img_size)
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy']
              )

In [ ]:
x_train2.shape

In [ ]:
history = model.fit(x_train2,
                    y_train2,
                    batch_size=24,
                    epochs=100,
                    steps_per_epoch=x_train2.shape[0] // 24,
                    verbose=0,
                    callbacks=[ES_monitor,LR_reduce],
                    validation_data=(x_val, y_val),
                    validation_steps=x_val.shape[0]//24
                    )

Lets print our results

In [ ]:
score = model.evaluate(x_val, y_val, verbose=0)

print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
plot_history(history)

Our accuracy has improved to 87%, but there are still some problems, we can see at the end that there is still a gap between the validation and the train matrices, which can indicate that there is some kind of overfitting. We can also see that the model is still having hard time classifing the multi-diseased class, due to it lower AUC-ROC score.
So where did the improvment came from?





Lets take a look at the confusin matrix for more insight

In [ ]:
y_val_pred = model.predict_classes(x_val)
mat = sklearn.metrics.confusion_matrix(get_target_array_no_im_id(y_val),y_val_pred)
sns.heatmap(mat.T, square=True, annot=True, cbar=False)
matplotlib.pyplot.xlabel('true label')
matplotlib.pyplot.ylabel('predict label');

The classes are ordered in this manner: "healthy"  "multiple_diseases"  "rust"  "scab". We can see that our  new model is pretty good at identifing the "rust" class and that it is over all can identify the classes we a decent accuracy as did the last one, but here we can see that using a better rsoluton made the model to identify the "scab" leaves better, as it can easily differ the leaf veins from the disease symptoms. We can see that the main issue of our new model has retied it poor prediction of the "multiple_diseases" class, it classifies those images as having one of the diseases and not both. It could be that the underrepresentation of that class is a big factor of that. 

In [ ]:
model.save("..output/kaggle/working/model_224X224.h5")

### **Data Transformations and Augmentation**

Image transformations and augmentation is an efficient way to diversify data and to generalize a model, we will try to use it to improve our model predictions and to maybe aid it to classify the "multiple_diseases" class better, as generating more images could give it a greater number of relevant images 

We will use a built in function to do so, and we will change images rotation, orientation, brightness and scale

 

In [ ]:
datagen = keras.preprocessing.image.ImageDataGenerator(rotation_range=45,
                             shear_range=.25,
                              zoom_range=.25,
                              width_shift_range=.25,
                              height_shift_range=.25,
                              rescale=1/255,
                              brightness_range=[.5,1.5],
                              horizontal_flip=True,
                              vertical_flip=True,
                              fill_mode='nearest'
#                              featurewise_center=True,
#                              samplewise_center=True,
#                              featurewise_std_normalization=True,
#                              samplewise_std_normalization=True,
#                              zca_whitening=True
                              )

Running the model on 224x224 images with the transformations and augmentation

In [ ]:
model = creat_model(img_size)
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy']
              )

In [ ]:
history = model.fit_generator(datagen.flow(x_train2, y_train2, batch_size=24),
                              epochs=100,
                              steps_per_epoch=x_train2.shape[0] // 24,
                              verbose=0,
                              callbacks=[ES_monitor,LR_reduce],
                              validation_data=datagen.flow(x_val, y_val,batch_size=24),
                              validation_steps=x_val.shape[0]//24
                              )

Lets print our results

In [ ]:
score = model.evaluate(x_val, y_val, verbose=0)

print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
plot_history(history)

Our accuracy has improved to 92%, a great accomplishment. we can see that the vlidation and test graph have no gap which can show that the model didn't overpitted to the test. We can see that the model is having a better time classifing the "multiple_diseases" class from the better AUC-ROC score. 

Lets take a look at the confusin matrix for more insight

In [ ]:
y_val_pred = model.predict_classes(x_val)
mat = sklearn.metrics.confusion_matrix(get_target_array_no_im_id(y_val),y_val_pred)
sns.heatmap(mat.T, square=True, annot=True, cbar=False)
matplotlib.pyplot.xlabel('true label')
matplotlib.pyplot.ylabel('predict label');

The classes are ordered in this manner: "healthy" "multiple_diseases" "rust" "scab". We can see that our new model is better at idintifing the "healthy" "rust" "scab" classes, as it classified corretly almost all of the images of those classes, but still even though is classified more images of that class correctly, the classification seems random.  Again it could be that the underrepresentation of that class is a big factor of that.

In [ ]:
model.save("..output/kaggle/working/model_224X224_image_generate.h5")

### **Handling Imbalanced in Our Dataset -SMOTE**


We'll be applying Synthetic Minority Oversampling Technique (SMOTE). SMOTE works by selecting examples that are close in the feature space, drawing a line between the examples in the feature space and drawing a new sample at a point along that line, and by that balance a minority class. [SMOTE](https://machinelearningmastery.com/smote-oversampling-for-imbalanced-classification/)


 Lets check its affect on our dataset. The size of our dataset before running the function

In [ ]:
print(x_train2.shape,y_train2.shape)

Applying SMOTE on our dataset

In [ ]:
 sm = imblearn.over_sampling.SMOTE(random_state = 115) 
 
x_train2, y_train2 = sm.fit_resample(get_all_pixels(x_train2),y_train2.to_numpy())
x_train2 = x_train2.reshape((-1, img_size, img_size, 3))
x_train2.shape, y_train2.sum(axis=0)

We see that we have added about 500 examples,most of them to the minority class, and our data is now balanced

Let's try to fit the CNN to the new dataset

In [ ]:
model = creat_model(img_size)
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy']
              )

In [ ]:
history = model.fit_generator(datagen.flow(x_train2, y_train2, batch_size=24),
                              epochs=250,
                              steps_per_epoch=x_train2.shape[0] // 24,
                              verbose=0,
                              callbacks=[ES_monitor,LR_reduce],
                              validation_data=datagen.flow(x_val, y_val,batch_size=24),
                              validation_steps=x_val.shape[0]//24
                              )

Lets print our results

In [ ]:
score = model.evaluate(x_val, y_val, verbose=0)

print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
plot_history(history)

Our accuracy has decreased to 74%, Worst then our first CNN. It hard to understand what went wrong; so lets take a look at the confusin matrix for more insight

In [ ]:
y_val_pred = model.predict_classes(x_val)
mat = sklearn.metrics.confusion_matrix(get_target_array_no_im_id(y_val),y_val_pred)
sns.heatmap(mat.T, square=True, annot=True, cbar=False)
matplotlib.pyplot.xlabel('true label')
matplotlib.pyplot.ylabel('predict label');

The classes are ordered in this manner: "healthy" "multiple_diseases" "rust" "scab". We can see that our new model classified most of the "multiple_diseases" class, but by doing it, it also classified leaves that are in the "rust" class as "multiple_diseases", and that has made it worse. It seems that the fact that the "multiple_diseases" has similar features of the other classes made the "synthetic" samlpes have similar atribute to this classes which made the model to get worse. 

In [ ]:
model.save("..output/kaggle/working/model_224X224_smoth.h5")

## **Transfer Learning**

The general idea of transfer learning is to use knowledge learned from tasks for which a lot of labelled data is available in settings where only little labelled data is available. We will use the pretrained model DenseNet121 inorder to classify

### **Densenet Model**

We chose DenseNet because it have some advantages over CNN when they are deep. This is because the path for information from the input layer until the output layer (and for the gradient in the opposite direction) becomes so big, that they can get vanished before reaching the other side.
DenseNets simplify the connectivity pattern between layers introduced in other architectures:
Highway Networks [2]
Residual Networks [3]
Fractal Networks [4]
This solve the problem ensuring maximum information (and gradient) flow. To do it, the model is simply connected from every layer directly to the other ones. Instead of drawing representational power from extremely deep or wide architectures, DenseNets exploit the potential of the network through feature reuse. Counter-intuitively, by connecting this way DenseNets require fewer parameters than an equivalent traditional CNN, as there is no need to learn redundant feature maps

We will start by setting a TPU

In [ ]:
AUTO = tf.data.experimental.AUTOTUNE
tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
strategy = tf.distribute.experimental.TPUStrategy(tpu)
warnings.filterwarnings("ignore")

Now we will load the images and split them to train and validation

In [ ]:
img_size = 224

In [ ]:
train_224X224_images = train_data["image"].progress_apply(load_image, args=((img_size,img_size),))

In [ ]:
current_train_data=train_data.loc[train_224X224_images.index]
images_array=images_4d_array(train_224X224_images)
targets=current_train_data.drop('image',axis=1)
x_train2, x_val, y_train2, y_val = sklearn.model_selection.train_test_split(images_array, targets, test_size=0.2, random_state=seed)
x_train2 = x_train2.astype('float32')
x_val = x_val.astype('float32')
x_train2 /= 255
x_val /= 255
x_train2.shape, x_val.shape, y_train2.shape, y_val.shape

Now we will define the model and try to fit it

In [ ]:
with strategy.scope():
    model = tf.keras.Sequential([tf.keras.applications.DenseNet121(input_shape=(224, 224, 3),
                                             weights='imagenet',
                                             include_top=False),
                                 L.GlobalAveragePooling2D(),
                                 L.Dense(y_train2.shape[1],
                                         activation='softmax')])
        
    model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy']
              )
    model.summary()

In [ ]:
history = model.fit(x_train2,y_train2,
                    epochs=250,
                    callbacks=[ES_monitor,LR_reduce],
                    steps_per_epoch=15,
                    verbose=0,
                    validation_data=(x_val, y_val))

Lets print our results

In [ ]:
score = model.evaluate(x_val, y_val, verbose=0)

print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
plot_history(history)

Our accuracy has stayed 92%, and the fitting was much faster, but there are still some problems, we can see at the end that there i a gap between the validation and the train matrices, which can indicate that there is some kind of overfittin, the accuracy of the train data is almos perferct, which can show again over fitting, but we can see that this is the case from almot the beggining which indicate that it could have been because of the pre trained model and it's proficiency  

Lets take a look at the confusin matrix for more insight

In [ ]:
y_val_pred = model.predict_classes(x_val)
mat = sklearn.metrics.confusion_matrix(get_target_array_no_im_id(y_val),y_val_pred)
sns.heatmap(mat.T, square=True, annot=True, cbar=False)
matplotlib.pyplot.xlabel('true label')
matplotlib.pyplot.ylabel('predict label');

The classes are ordered in this manner: "healthy" "multiple_diseases" "rust" "scab". We can see that our model is pretty accurate at idintifing the "healthy" "rust" "scab" classes, as it classified corretly almost all of the images of those classes. Again the "multiple_diseases" class images' classification is not better then random. It could be that the underrepresentation of that class is a big factor of that and we haven't found a model that whould help us with that.

In [ ]:
model.save("..output/kaggle/working/model_densnet121.h5")


### **DenseNet With SMOTE**

Our last attempt with SMOTE left us with mixed feeling about it, but it did had an impact on the "multiple_diseases" class, was the wannted one. So we will try it once more but with DenseNet this time 

Applying SMOTE on our dataset

In [ ]:
sm = imblearn.over_sampling.SMOTE(random_state = 115) 
 
x_train2, y_train2 = sm.fit_resample(get_all_pixels(x_train2),y_train2.to_numpy())
x_train2 = x_train2.reshape((-1, img_size, img_size, 3))
x_train2.shape, y_train2.sum(axis=0)

Now we will define the model and try to fit it

In [ ]:
with strategy.scope():
    model = tf.keras.Sequential([tf.keras.applications.DenseNet121(input_shape=(224, 224, 3),
                                             weights='imagenet',
                                             include_top=False),
                                 L.GlobalAveragePooling2D(),
                                 L.Dense(y_train2.shape[1],
                                         activation='softmax')])
        
    model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy']
              )
    model.summary()

In [ ]:
history = model.fit(x_train2,y_train2,
                    epochs=250,
                    callbacks=[ES_monitor,LR_reduce],
                    steps_per_epoch=15,
                    verbose=0,
                    validation_data=(x_val, y_val))

Lets print our results

In [ ]:
score = model.evaluate(x_val, y_val, verbose=0)

print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
plot_history(history)

Our accuracy has stayed 92%, but there are the same problems, we can see at the end that there is a gap between the validation and the train matrices, which can indicate that there is some kind of overfittin, the accuracy of the train data is almos perferct, which can show again over fitting, but we can see that this is the case from almot the beggining which indicate that it could have been because of the pre trained model and it's proficiency

Lets take a look at the confusin matrix for more insight

In [ ]:
y_val_pred = model.predict_classes(x_val)
mat = sklearn.metrics.confusion_matrix(get_target_array_no_im_id(y_val),y_val_pred)
sns.heatmap(mat.T, square=True, annot=True, cbar=False)
matplotlib.pyplot.xlabel('true label')
matplotlib.pyplot.ylabel('predict label');

The classes are ordered in this manner: "healthy" "multiple_diseases" "rust" "scab". We can see that there were minimal change to the classification and that SMOTE did't help us classify the "multiple_diseases" better

In [ ]:
model.save("..output/kaggle/working/model_densnet121_SMOTE.h5")

### **DenseNet With Data Transformations and Augmentation**

As using DenseNet with SMOTE didn't produced a better model, we will use Data Transformations and Augmentation as well. Maybe that could produce a better model. We Didn't use it in at first place because using TPU and ImageDataGenerator is no possible, and using a GPU take considerebly longer.

Loading the images and spliting them to train and test

In [ ]:
img_size = 224

In [ ]:
train_224X224_images = train_data["image_id"].progress_apply(load_image, args=((img_size,img_size),))

In [ ]:
current_train_data=train_data.loc[train_224X224_images.index]
images_array=images_4d_array(train_224X224_images)
targets=current_train_data.drop('image_id',axis=1)
x_train2, x_val, y_train2, y_val = sklearn.model_selection.train_test_split(images_array, targets, test_size=0.2, random_state=seed)
x_train2 = x_train2.astype('float32')
x_val = x_val.astype('float32')
x_train2 /= 255
x_val /= 255
x_train2.shape, x_val.shape, y_train2.shape, y_val.shape

Now we will define the model and try to fit it

In [ ]:

model = tf.keras.Sequential([tf.keras.applications.DenseNet121(input_shape=(224, 224, 3),
                                          weights='imagenet',
                                          include_top=False),
                              L.GlobalAveragePooling2D(),
                              L.Dense(y_train2.shape[1],
                                      activation='softmax')])
    
model.compile(optimizer='rmsprop',
          loss='categorical_crossentropy',
          metrics=['accuracy']
          )
model.summary()

In [ ]:
history = model.fit(datagen.flow(x_train2,y_train2,batch_size=24),
                    epochs=250,
                    callbacks=[ES_monitor,LR_reduce],
                    steps_per_epoch=15,
                    verbose=0,
                    validation_data=(x_val, y_val))

Lets print our results

In [ ]:
score = model.evaluate(x_val, y_val, verbose=0)

print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
plot_history(history)

Our accuracy has improved to 94%, the best accuracy thus far we can see that the vlidation and test graph have no gap which can show that the model didn't overpitted to the test. We can see that the model is having a better time classifing the "multiple_diseases" class then ever before from the better AUC-ROC score.

Lets take a look at the confusin matrix for more insight

In [ ]:
y_val_pred = model.predict_classes(x_val)
mat = sklearn.metrics.confusion_matrix(get_target_array_no_im_id(y_val),y_val_pred)
sns.heatmap(mat.T, square=True, annot=True, cbar=False)
matplotlib.pyplot.xlabel('true label')
matplotlib.pyplot.ylabel('predict label');

The classes are ordered in this manner: "healthy" "multiple_diseases" "rust" "scab". We can see that our new model is better at idintifing the "multiple_diseases" class while classifing the other classes, then any other model we produced. It main problem is still this class but it does much better job at classifing it.

In [ ]:
model.save("..output/kaggle/working/model_densnet121_DG.h5")


### **DenseNet With Data Transformations and Augmentation and With  SMOTE**

Our last model did great, but still can have difficulties in classifing the "multiple_diseases". Even though SMOTE has faild us one, and had no effect the other time, e will try to use SMOTE to better our model, we don't have great hopes for it to work but maybe it wiil prove us wrong 

Again we will use the same procider as before but we will use SMOTE as well

Applying SMOTE on our dataset

In [ ]:
sm = imblearn.over_sampling.SMOTE(random_state = 115) 
 
x_train2, y_train2 = sm.fit_resample(get_all_pixels(x_train2),y_train2.to_numpy())
x_train2 = x_train2.reshape((-1, img_size, img_size, 3))
x_train2.shape, y_train2.sum(axis=0)

Now we will define the model and try to fit it

In [ ]:

model = tf.keras.Sequential([tf.keras.applications.DenseNet121(input_shape=(224, 224, 3),
                                          weights='imagenet',
                                          include_top=False),
                              L.GlobalAveragePooling2D(),
                              L.Dense(y_train2.shape[1],
                                      activation='softmax')])
    
model.compile(optimizer='rmsprop',
          loss='categorical_crossentropy',
          metrics=['accuracy']
          )
model.summary()

In [ ]:
history = model.fit(datagen.flow(x_train2,y_train2,batch_size=24),
                    epochs=250,
                    callbacks=[ES_monitor,LR_reduce],
                    steps_per_epoch=15,
                    verbose=0,
                    validation_data=(x_val, y_val))

Lets print our results

In [ ]:
score = model.evaluate(x_val, y_val, verbose=0)

print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
plot_history(history)

SMOTE didn't fail us and didn't help us. our accuracy has decreased to 88%, not the worst but a downgarde from our privious model. lets take a look at the confusin matrix for more insight about what went wrong

In [ ]:
y_val_pred = model.predict_classes(x_val)
mat = sklearn.metrics.confusion_matrix(get_target_array_no_im_id(y_val),y_val_pred)
sns.heatmap(mat.T, square=True, annot=True, cbar=False)
matplotlib.pyplot.xlabel('true label')
matplotlib.pyplot.ylabel('predict label');

The classes are ordered in this manner: "healthy" "multiple_diseases" "rust" "scab". We can see that our new model is worse then the one before at any classification. We can see that the SMOTE didnt had the effect that we wanted and it had all the negative effects from the first time we used it

In [ ]:
model.save("..output/kaggle/working/model_densnet121_SMOTE_DG.h5")


# **Discussion**

Our goal in this project was to produce a model that can classify leaves images to 4 classes of health condition. The data set we had wes imbalance, one of the classes had drastically less images. Throughout the project that fact had great influence on our models and success. At first we explored our data and noticed that there is color difference between classes, and thought that it could heip us, but as the PCA showed us, the color of the image is greatly infuenced by the background and it has a greater effect then the symptoms of the diseases on the leaves. We tried to negate this  effect with 2 methods, cropping the leaves out ot the image and by blacking out the background, non of those methods were successful. Per image we can use the methods to great extent, but automatin those methods prove too sofisticated for us, because of the color of the background compare to the leaves, because of the curved shape of the leave which made their edges lose focus in some instances, which meant that their veins were more prominent as an edge.

As of next we tried some classic prediction models, which didn't go good at all, and gave prediction that are good as guessing.

Next we tried the the most common method for classifing images, neural network. At first we tried a basic model that didn't performd well at all, next we continued to convolutional neural network, a great method for image classifing, that wes the first model that gave us good results. We tried at first to use it on images with the size of 100X100 pixels, it had good results but it misclassified some diseased infected leave as healthy, but more noticeably it didn't classified the "multiple_diseases" class well at all, due to it being underrepresented and having features of other classes. Using bigger images, 224X244 pixels to be specific, we managed to reduce it's misclassification of the diseased leaves as healthy, but didn't change the "multiple_diseases" problem, we saw as well some signs of overfiting to the train set. 

As we saw the problems in our previous model, we use data augmentatinons, we changed images rotation, orientation, brightness and scale, and fed them to the CNN. This method improved our model and msde his be more presice when classifing diseased leaves, but once again we colud not classify the "multiple_diseases" class well.

We tried next a method that could maybe help us with this problem, we used SMOTE, it's a method that balances our data by generating samples to the minor class, but as it turn out this method just made the model worse. The problem was that our "multiple_diseases" class has similar featurs as the other two diseases calsses, and by generating the samples it made those features ones that assosiate greatly with the "multiple_diseases" class, and not the other diseased classes.

Thus far our best attempt ,data augmentation, was pretty good but had some difficulties and could not tackle the main issue of the data set, it's imbalance. 

Next we tried to incorporate transfer learning to our attempts. This methos is using an allready traind model on general case as our base model, and in addition we used a different neural network, denstenet which has different kind of architecture the our CNN. We tried using it with or without SMOTE and with or without data augmentation. The best model was produced as a result of the the one with data augmentation and without SMOTE. It can tackle the "multiple_diseases" class much better but still has a bit of dificult with this class, at the rest it does great work.


Our data main challenges were it's imbalance and the fact that the leaves were filmed on a dinamic background. Further more, another challenge is the resemblance of some of its classes features. All of the above made our work interesting and challenging. We could not find an efficient way to reduce the impact that the back ground has, and could not find a way that could heip us with the imbalances without reducing model accurecy. The first problem could be solved by creating a model that can by himself black out the background, but that problem is another big one that would required a project by itself. The secound problem could be tackled by gathering more samples of this class, a mission for plant disease expert that can identify the correct disease. 

Overall we produced a competant model that can in great success predict the disease an apple leaf has.



# **Conclusion**

This project was a great challenge but a great fun to work on. We've learned a lot and were able to dive into the world of data science. We learned that some task that look trivial like identifing a leaf and differentiating it from the background can be convoluted if the background has certain features. We also learnd that in the case that we have two classes with shared features using sample genaration would probebly wont work. We saw first-handed how promenent of a  solution neural networks for classifing images, especially over classic methods. We found out that managing resources is a big part of a data science project and can have big impect on our time and effort.